<a href="https://colab.research.google.com/github/shamrutha16/Card_Sheild/blob/shamrutha16-patch-1/EDUASSISTBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install transformers torch nltk gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [3]:
# Import necessary libraries
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import random
import re
import gradio as gr
from datetime import datetime, timedelta
import json


In [4]:

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
# =============================================================================
# INTENT CLASSIFICATION SETUP
# =============================================================================

class IntentClassifier:
    def __init__(self):
        # Initialize DistilBERT tokenizer and model
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        self.model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

        # Define intents and their keywords
        self.intents = {
            'course_recommendation': [
                'course', 'class', 'recommend', 'suggestion', 'major', 'minor',
                'elective', 'prerequisite', 'curriculum', 'schedule', 'registration'
            ],
            'deadlines': [
                'deadline', 'due date', 'last day', 'cutoff', 'registration deadline',
                'assignment', 'project', 'exam', 'final', 'drop', 'withdraw'
            ],
            'mental_health': [
                'mental health', 'stress', 'anxiety', 'depression', 'counseling',
                'therapy', 'wellness', 'self-care', 'burnout', 'overwhelmed',
                'help', 'support', 'crisis'
            ],
            'campus_events': [
                'event', 'activity', 'club', 'organization', 'meeting', 'workshop',
                'seminar', 'lecture', 'social', 'networking', 'campus life'
            ],
            'faq': [
                'how to', 'what is', 'where is', 'when is', 'why', 'help',
                'question', 'information', 'guide', 'tutorial'
            ]
        }

        # Initialize text preprocessing
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))


In [8]:
def preprocess_text(self, text):
        """Clean and preprocess input text"""
        # Convert to lowercase
        text = text.lower()

        # Remove special characters and numbers
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords and lemmatize
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens
                 if token not in self.stop_words and len(token) > 2]

        return ' '.join(tokens)

def classify_intent(self, text):
        """Classify user intent using keyword matching and NLP"""
        processed_text = self.preprocess_text(text)

        # Keyword-based classification
        text_words = processed_text.split()

        # Calculate scores for each intent
        intent_scores = {}
        for intent, keywords in self.intents.items():
            score = sum(1 for keyword in keywords if keyword in text_words)
            intent_scores[intent] = score

        # Get the intent with highest score
        if max(intent_scores.values()) > 0:
            return max(intent_scores, key=intent_scores.get)
        else:
            return 'general'

In [9]:

# =============================================================================
# RESPONSE GENERATOR
# =============================================================================

class ResponseGenerator:
    def __init__(self):
        # Sample responses for each intent
        self.responses = {
            'course_recommendation': [
                "🎓 Based on your interests, I'd recommend checking out courses in Computer Science, Psychology, or Business! What's your major?",
                "📚 I can help you find the perfect courses! Are you looking for core requirements or electives?",
                "🔍 Let me suggest some courses that align with your academic goals. What subjects interest you most?",
                "📖 For course recommendations, I'd need to know your major and interests. What field are you studying?"
            ],
            'deadlines': [
                "⏰ Important deadlines coming up! Registration closes on March 15th, and finals week starts May 1st.",
                "📅 Don't forget! The last day to drop classes is April 10th, and graduation applications are due by March 30th.",
                "⏳ Key dates to remember: Spring break is March 10-17, and summer registration opens April 1st.",
                "🗓️ Here are your upcoming deadlines: Midterms next week, project submissions due Friday!"
            ],
            'mental_health': [
                "�� Your mental health matters! The campus counseling center offers free sessions. Would you like their contact info?",
                "🧠 It's completely normal to feel overwhelmed. The wellness center has great resources and support groups.",
                "💚 Remember, it's okay to ask for help. The student health services offer 24/7 crisis support.",
                "🌱 Take care of yourself! Consider joining a mindfulness workshop or talking to a counselor."
            ],
            'campus_events': [
                "🎉 Exciting events this week! There's a career fair on Wednesday and a movie night on Friday!",
                "🎪 Check out the student activities board! This month we have guest speakers, workshops, and social mixers.",
                "🎭 Don't miss the spring festival this weekend! Live music, food trucks, and fun activities for everyone.",
                "🎨 This week's highlights: Art exhibit opening, poetry slam, and intramural sports tournaments!"
            ],
            'faq': [
                "❓ I'd be happy to help! Could you please provide more details about your question?",
                "🤔 That's a great question! Let me find the most accurate information for you.",
                "💡 I can help with that! What specific information are you looking for?",
                "�� I'd love to assist you! Can you clarify what you need help with?"
            ],
            'general': [
                "👋 Hi there! I'm EDUASSISTBOT, your college support assistant. How can I help you today?",
                "🎓 Welcome! I'm here to help with course info, deadlines, campus events, and more. What do you need?",
                "💬 Hello! I can assist with academic questions, mental health resources, and campus life. What's on your mind?",
                "🌟 Hi! I'm your friendly college assistant. Feel free to ask me anything about student life!"
            ]
        }

        # Contextual responses based on conversation history
        self.context_responses = {
            'greeting': [
                "👋 Hello again! How else can I assist you today?",
                "😊 Great to see you! What other questions do you have?",
                "🎯 Welcome back! Is there something specific you'd like to know?"
            ],
            'thanks': [
                "You're very welcome! 😊 Is there anything else I can help you with?",
                "Happy to help! 🌟 Don't hesitate to ask if you need more assistance.",
                "Anytime! �� Feel free to reach out whenever you have questions."
            ]
        }

    def generate_response(self, intent, user_input, chat_history):
        """Generate appropriate response based on intent and context"""
        # Check for contextual responses
        if any(word in user_input.lower() for word in ['hello', 'hi', 'hey', 'good morning', 'good afternoon']):
            return random.choice(self.context_responses['greeting'])

        if any(word in user_input.lower() for word in ['thanks', 'thank you', 'appreciate it']):
            return random.choice(self.context_responses['thanks'])

        # Generate intent-based response
        if intent in self.responses:
            response = random.choice(self.responses[intent])

            # Add contextual information based on chat history
            if intent == 'deadlines' and len(chat_history) > 0:
                response += " I can also check your specific course deadlines if you'd like!"
            elif intent == 'course_recommendation' and len(chat_history) > 0:
                response += " Based on our conversation, I think you might enjoy these options!"

            return response
        else:
            return random.choice(self.responses['general'])

In [10]:
# =============================================================================
# MAIN CHATBOT CLASS
# =============================================================================

class EDUASSISTBOT:
    def __init__(self):
        self.intent_classifier = IntentClassifier()
        self.response_generator = ResponseGenerator()
        self.chat_history = []

        # Bot personality and capabilities
        self.bot_info = {
            'name': 'EDUASSISTBOT',
            'version': '2.0',
            'capabilities': [
                'Course recommendations and academic guidance',
                'Deadline reminders and important dates',
                'Mental health resources and wellness support',
                'Campus events and student activities',
                'General FAQs and student support'
            ]
        }

    def process_message(self, user_input):
        """Process user input and generate response"""
        if not user_input.strip():
            return "Please type something so I can help you! 😊"

        # Classify intent
        intent = self.intent_classifier.classify_intent(user_input)

        # Generate response
        response = self.response_generator.generate_response(intent, user_input, self.chat_history)

        # Update chat history
        timestamp = datetime.now().strftime("%H:%M")
        self.chat_history.append({
            'timestamp': timestamp,
            'user': user_input,
            'bot': response,
            'intent': intent
        })

        return response

    def get_chat_history(self):
        """Get formatted chat history for display"""
        formatted_history = []
        for entry in self.chat_history:
            formatted_history.append(f"**{entry['timestamp']}** - You: {entry['user']}")
            formatted_history.append(f"**{entry['timestamp']}** - {self.bot_info['name']}: {entry['response']}")
        return "\n".join(formatted_history)

    def clear_history(self):
        """Clear chat history"""
        self.chat_history = []
        return "Chat history cleared! 🗑️"

In [11]:
# =============================================================================
# GRADIO WEB INTERFACE
# =============================================================================

def create_interface():
    """Create the Gradio web interface"""
    # Initialize chatbot
    chatbot = EDUASSISTBOT()

    # Custom CSS for modern, themed interface
    css = """
    .gradio-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        min-height: 100vh;
        padding: 20px;
    }
    .main-container {
        background: white;
        border-radius: 20px;
        box-shadow: 0 20px 40px rgba(0,0,0,0.1);
        padding: 30px;
        max-width: 800px;
        margin: 0 auto;
    }
    .header {
        text-align: center;
        margin-bottom: 30px;
        color: #4a5568;
    }
    .header h1 {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        font-size: 2.5em;
        font-weight: bold;
        margin-bottom: 10px;
    }
    .header p {
        color: #718096;
        font-size: 1.1em;
    }
    .chat-container {
        border: 2px solid #e2e8f0;
        border-radius: 15px;
        padding: 20px;
        background: #f7fafc;
        margin-bottom: 20px;
        min-height: 400px;
        max-height: 500px;
        overflow-y: auto;
    }
    .input-container {
        display: flex;
        gap: 10px;
        margin-bottom: 20px;
    }
    .input-box {
        flex: 1;
        border: 2px solid #e2e8f0;
        border-radius: 10px;
        padding: 12px;
        font-size: 16px;
    }
    .send-btn {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        border-radius: 10px;
        padding: 12px 24px;
        font-size: 16px;
        cursor: pointer;
        transition: transform 0.2s;
    }
    .send-btn:hover {
        transform: translateY(-2px);
    }
    .clear-btn {
        background: #e53e3e;
        color: white;
        border: none;
        border-radius: 10px;
        padding: 12px 24px;
        font-size: 16px;
        cursor: pointer;
        transition: transform 0.2s;
    }
    .clear-btn:hover {
        transform: translateY(-2px);
    }
    .chat-message {
        margin-bottom: 15px;
        padding: 10px 15px;
        border-radius: 10px;
        max-width: 80%;
    }
    .user-message {
        background: #667eea;
        color: white;
        margin-left: auto;
        text-align: right;
    }
    .bot-message {
        background: #f1f5f9;
        color: #1a202c;
        border: 1px solid #e2e8f0;
    }
    """

    # Interface components
    with gr.Blocks(css=css, title="EDUASSISTBOT - College Support Assistant") as interface:
        with gr.Column(elem_classes="main-container"):
            # Header
            with gr.Column(elem_classes="header"):
                gr.Markdown("# 🎓 EDUASSISTBOT")
                gr.Markdown("### Your Intelligent College Support Assistant")
                gr.Markdown("*Powered by NLP and AI to help with academic guidance, mental health resources, campus events, and more!*")

            # Chat display
            chat_display = gr.HTML(
                value="<div class='chat-container'><p style='text-align: center; color: #718096;'>Start a conversation with EDUASSISTBOT! 👋</p></div>",
                elem_classes="chat-container"
            )

            # Input section
            with gr.Row():
                user_input = gr.Textbox(
                    placeholder="Ask me anything about college life, courses, deadlines, or campus events...",
                    label="Your Message",
                    elem_classes="input-box"
                )
                send_btn = gr.Button("Send 💬", elem_classes="send-btn")

            # Control buttons
            with gr.Row():
                clear_btn = gr.Button("Clear Chat 🗑️", elem_classes="clear-btn")
                gr.Markdown("**Capabilities:** Course recommendations • Deadline reminders • Mental health resources • Campus events • General FAQs")

            # Chatbot functionality
            def chat_with_bot(message):
                if not message.strip():
                    return chat_display.value

                # Get bot response
                response = chatbot.process_message(message)

                # Format chat display
                current_chat = chat_display.value
                if "Start a conversation" in current_chat:
                    current_chat = ""

                timestamp = datetime.now().strftime("%H:%M")
                new_message = f"""
                <div class='chat-message user-message'>
                    <strong>{timestamp}</strong> - You: {message}
                </div>
                <div class='chat-message bot-message'>
                    <strong>{timestamp}</strong> - EDUASSISTBOT: {response}
                </div>
                """

                return current_chat + new_message

            def clear_chat():
                chatbot.clear_history()
                return "<div class='chat-container'><p style='text-align: center; color: #718096;'>Chat cleared! Start a new conversation with EDUASSISTBOT! 👋</p></div>"

            # Event handlers
            send_btn.click(
                fn=chat_with_bot,
                inputs=[user_input],
                outputs=[chat_display]
            )

            user_input.submit(
                fn=chat_with_bot,
                inputs=[user_input],
                outputs=[chat_display]
            )

            clear_btn.click(
                fn=clear_chat,
                outputs=[chat_display]
            )

    return interface


In [ ]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    print("🚀 Initializing EDUASSISTBOT...")
    print("📚 Setting up NLP components...")
    print("🎨 Creating web interface...")

    # Create and launch the interface
    interface = create_interface()

    print("✅ EDUASSISTBOT is ready!")
    print("🌐 Launching web interface...")
    print("�� Use the public URL to access EDUASSISTBOT from anywhere!")

    # Launch with public sharing enabled
    interface.launch(share=True, debug=True)

🚀 Initializing EDUASSISTBOT...
📚 Setting up NLP components...
🎨 Creating web interface...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ EDUASSISTBOT is ready!
🌐 Launching web interface...
�� Use the public URL to access EDUASSISTBOT from anywhere!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c9a0be263464c4abc8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
